# LLC matched to MODIS -- Developing

In [17]:
# imports
from importlib import reload

import os
import numpy as np
import datetime

import pandas
import xarray
import h5py

from matplotlib import pyplot as plt
import seaborn as sns

from ulmo import io as ulmo_io
from ulmo import plotting
from ulmo.llc import io as llc_io
from ulmo.llc import plotting as llc_plotting

from ulmo.utils import image_utils
from ulmo.analysis import figures as ulmo_figs
from ulmo.models import io as model_io
from ulmo.modis import io as modis_io

from astropy import units
from astropy.coordinates import SkyCoord, match_coordinates_sky


# Init

In [2]:
field_size=(64,64)

In [3]:
CC_max = 1e-4

# Load

In [7]:
reload(llc_io)
CC_mask = llc_io.load_CC_mask(field_size=field_size, 
                                  local=True)

Loading LLC CC mask from /home/xavier/Projects/Oceanography/data/LLC/CC/LLC_CC_mask_64.nc


In [8]:
CC_mask

<xarray.Dataset>
Dimensions:  (x: 12960, y: 17280)
Coordinates:
    lon      (x, y) float32 ...
    lat      (x, y) float32 ...
Dimensions without coordinates: x, y
Data variables:
    CC_mask  (x, y) float64 ...

# Coords

In [9]:
# Cut
good_CC = CC_mask.CC_mask.values < CC_max
good_CC_idx = np.where(good_CC)

# Build coords
llc_lon = CC_mask.lon.values[good_CC].flatten()
llc_lat = CC_mask.lat.values[good_CC].flatten()
print("Building LLC SkyCoord")
llc_coord = SkyCoord(llc_lon*units.deg + 180.*units.deg, 
                     llc_lat*units.deg, 
                     frame='galactic')

Building LLC SkyCoord


In [10]:
del CC_mask

# Times available

## Grab files

In [15]:
reload(ulmo_io)
llc_files = ulmo_io.list_of_bucket_files('llc', prefix='/ThetaUVSalt')

In [16]:
llc_files

['ThetaUVSalt/LLC4320_2011-09-13T00_00_00.nc',
 'ThetaUVSalt/LLC4320_2011-09-13T01_00_00.nc',
 'ThetaUVSalt/LLC4320_2011-09-13T02_00_00.nc',
 'ThetaUVSalt/LLC4320_2011-09-13T03_00_00.nc',
 'ThetaUVSalt/LLC4320_2011-09-13T04_00_00.nc',
 'ThetaUVSalt/LLC4320_2011-09-13T05_00_00.nc',
 'ThetaUVSalt/LLC4320_2011-09-13T06_00_00.nc',
 'ThetaUVSalt/LLC4320_2011-09-13T07_00_00.nc',
 'ThetaUVSalt/LLC4320_2011-09-13T08_00_00.nc',
 'ThetaUVSalt/LLC4320_2011-09-13T09_00_00.nc',
 'ThetaUVSalt/LLC4320_2011-09-13T10_00_00.nc',
 'ThetaUVSalt/LLC4320_2011-09-13T11_00_00.nc',
 'ThetaUVSalt/LLC4320_2011-09-13T12_00_00.nc',
 'ThetaUVSalt/LLC4320_2011-09-13T13_00_00.nc',
 'ThetaUVSalt/LLC4320_2011-09-13T14_00_00.nc',
 'ThetaUVSalt/LLC4320_2011-09-13T15_00_00.nc',
 'ThetaUVSalt/LLC4320_2011-09-13T16_00_00.nc',
 'ThetaUVSalt/LLC4320_2011-09-14T00_00_00.nc',
 'ThetaUVSalt/LLC4320_2011-09-14T12_00_00.nc',
 'ThetaUVSalt/LLC4320_2011-09-15T00_00_00.nc',
 'ThetaUVSalt/LLC4320_2011-09-15T12_00_00.nc',
 'ThetaUVSalt

## Times

In [30]:
times = [os.path.basename(ifile)[8:-3].replace('_',':') for ifile in llc_files]
times

['2011-09-13T00:00:00',
 '2011-09-13T01:00:00',
 '2011-09-13T02:00:00',
 '2011-09-13T03:00:00',
 '2011-09-13T04:00:00',
 '2011-09-13T05:00:00',
 '2011-09-13T06:00:00',
 '2011-09-13T07:00:00',
 '2011-09-13T08:00:00',
 '2011-09-13T09:00:00',
 '2011-09-13T10:00:00',
 '2011-09-13T11:00:00',
 '2011-09-13T12:00:00',
 '2011-09-13T13:00:00',
 '2011-09-13T14:00:00',
 '2011-09-13T15:00:00',
 '2011-09-13T16:00:00',
 '2011-09-14T00:00:00',
 '2011-09-14T12:00:00',
 '2011-09-15T00:00:00',
 '2011-09-15T12:00:00',
 '2011-09-16T00:00:00',
 '2011-09-16T12:00:00',
 '2011-09-17T00:00:00',
 '2011-09-17T12:00:00',
 '2011-09-18T00:00:00',
 '2011-09-18T12:00:00',
 '2011-09-19T00:00:00',
 '2011-09-19T12:00:00',
 '2011-09-20T00:00:00',
 '2011-09-20T12:00:00',
 '2011-09-21T00:00:00',
 '2011-09-21T12:00:00',
 '2011-09-22T00:00:00',
 '2011-09-22T12:00:00',
 '2011-09-23T00:00:00',
 '2011-09-23T12:00:00',
 '2011-09-24T00:00:00',
 '2011-09-24T12:00:00',
 '2011-09-25T00:00:00',
 '2011-09-25T12:00:00',
 '2011-09-26T00:

In [31]:
pandas.Timestamp(np.array(times))

TypeError: Cannot convert input [['2011-09-13T00:00:00' '2011-09-13T01:00:00' '2011-09-13T02:00:00'
 '2011-09-13T03:00:00' '2011-09-13T04:00:00' '2011-09-13T05:00:00'
 '2011-09-13T06:00:00' '2011-09-13T07:00:00' '2011-09-13T08:00:00'
 '2011-09-13T09:00:00' '2011-09-13T10:00:00' '2011-09-13T11:00:00'
 '2011-09-13T12:00:00' '2011-09-13T13:00:00' '2011-09-13T14:00:00'
 '2011-09-13T15:00:00' '2011-09-13T16:00:00' '2011-09-14T00:00:00'
 '2011-09-14T12:00:00' '2011-09-15T00:00:00' '2011-09-15T12:00:00'
 '2011-09-16T00:00:00' '2011-09-16T12:00:00' '2011-09-17T00:00:00'
 '2011-09-17T12:00:00' '2011-09-18T00:00:00' '2011-09-18T12:00:00'
 '2011-09-19T00:00:00' '2011-09-19T12:00:00' '2011-09-20T00:00:00'
 '2011-09-20T12:00:00' '2011-09-21T00:00:00' '2011-09-21T12:00:00'
 '2011-09-22T00:00:00' '2011-09-22T12:00:00' '2011-09-23T00:00:00'
 '2011-09-23T12:00:00' '2011-09-24T00:00:00' '2011-09-24T12:00:00'
 '2011-09-25T00:00:00' '2011-09-25T12:00:00' '2011-09-26T00:00:00'
 '2011-09-26T12:00:00' '2011-